In [1]:
import os
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline


In [2]:

PIPELINE_NAME = "pilcotech-obesity-pipeline"

# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
TUNER_MODULE_FILE = "modules/tuner.py"
# requirement_file = os.path.join(root, "requirements.txt")

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


Inisisasi Tfx pipeline komponent

In [3]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Tuner,
    Transform,
    Trainer,
    Evaluator,
    Pusher,
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy,
)


def init_components(
    data_dir,
    transform_module,
    training_module,
    tuner_module,
    training_steps,
    eval_steps,
    serving_model_dir,
):
    """Initiate tfx pipeline components

    Args:
        data_dir (str): a path to the data
        transform_module (str): a path to the transform_module
        training_module (str): a path to the transform_module
        training_steps (int): number of training steps
        eval_steps (int): number of eval steps
        serving_model_dir (str): a path to the serving model directory

    Returns:
        TFX components
    """
    output = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(
            splits=[
                example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
                example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
            ]
        )
    )

    example_gen = CsvExampleGen(input_base=data_dir, output_config=output)

    statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])

    schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_gen.outputs["schema"],
    )

    transform = Transform(
        examples=example_gen.outputs["examples"],
        schema=schema_gen.outputs["schema"],
        module_file=os.path.abspath(transform_module),
    )

    tuner = Tuner(
        module_file=os.path.abspath(tuner_module),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=training_steps),
        eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=eval_steps),
    )

    trainer = Trainer(
        module_file=os.path.abspath(training_module),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=training_steps),
        eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=eval_steps),
    )

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing),
    ).with_id("Latest_blessed_model_resolver")

    slicing_specs = [
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=["Gender_xf"]),
    ]

    metrics_specs = [
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(
                    class_name="SparseCategoricalAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.0001},
                        ),
                    ),
                ),
            ]
        )
    ]

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key="Label_xf")],
        slicing_specs=slicing_specs,
        metrics_specs=metrics_specs,
    )

    evaluator = Evaluator(
        examples=transform.outputs["transformed_examples"],
        model=trainer.outputs["model"],
        baseline_model=model_resolver.outputs["model"],
        eval_config=eval_config,
    )

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir
            )
        ),
    )

    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher,
    )

    return components


membuat module transform file

In [4]:
%%writefile {TRANSFORM_MODULE_FILE}

"""Transform module"""

import tensorflow as tf
import tensorflow_transform as tft

CATEGORICAL_FEATURES = {
    "Gender": 2,
}
NUMERICAL_FEATURES = ["Height", "Weight", "BMI", "Age"]
LABEL_KEY = "Label"


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


def convert_num_to_one_hot(label_tensor, num_labels=2):
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    # KATEGORIKAL
    for key in CATEGORICAL_FEATURES:
        dim = CATEGORICAL_FEATURES[key]
        int_value = tft.compute_and_apply_vocabulary(inputs[key], top_k=dim + 1)
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            int_value, num_labels=dim + 1
        )

    # NUMERIKAL
    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tf.expand_dims(
            tft.scale_to_0_1(inputs[feature]), -1
        )

    # LABEL
    outputs[transformed_name(LABEL_KEY)] = tft.compute_and_apply_vocabulary(
        inputs[LABEL_KEY], 4
    )

    return outputs


Overwriting modules/transform.py


membuat module tuner file

In [5]:
%%writefile {TUNER_MODULE_FILE}
# Import library
import tensorflow as tf
import keras_tuner as kt
import tensorflow_transform as tft

from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.components.tuner.component import TunerFnResult

from transform import CATEGORICAL_FEATURES, NUMERICAL_FEATURES, transformed_name
from trainer import input_fn


def model_builder(hp):
    """
    This function defines a Keras model and returns the model as a Keras object.
    """

    input_features = []

    for key in CATEGORICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(CATEGORICAL_FEATURES[key] + 1,), name=transformed_name(key))
        )

    for key in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(key))
        )

    concat = tf.keras.layers.concatenate(input_features)

    x = tf.keras.layers.Dense(
        hp.Choice('units_1', [128, 256, 512]),
        activation='relu'
    )(concat)
    x = tf.keras.layers.Dropout(hp.Choice('dropout_1', [0.2, 0.3, 0.4]))(x)

    x = tf.keras.layers.Dense(
        hp.Choice('units_2', [64, 128]),
        activation='relu'
    )(x)
    x = tf.keras.layers.Dropout(hp.Choice('dropout_2', [0.2, 0.3, 0.4]))(x)

    x = tf.keras.layers.Dense(
        hp.Choice('units_3', [32, 64]),
        activation='relu'
    )(x)
    x = tf.keras.layers.Dropout(hp.Choice('dropout_3', [0.2, 0.3, 0.4]))(x)

    num_classes = 4  # Ganti jika jumlah kelas berbeda
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model


def tuner_fn(fn_args: FnArgs):
    """
    Build the tuner and return TunerFnResult to TFX.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(
        fn_args.train_files,
        tf_transform_output,
        batch_size=32
    )

    eval_dataset = input_fn(
        fn_args.eval_files,
        tf_transform_output,
        batch_size=32
    )

    tuner = kt.RandomSearch(
        model_builder,
        objective='val_sparse_categorical_accuracy',
        max_trials=10,
        directory=fn_args.working_dir,
        project_name='tfx_tuner'
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_dataset,
            "validation_data": eval_dataset,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
            "epochs": 10,
            "callbacks": [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
        }
    )



Overwriting modules/tuner.py


membuat trainer module file

In [6]:
%%writefile {TRAINER_MODULE_FILE}

import os
import tensorflow as tf
import tensorflow_transform as tft
from keras.utils.vis_utils import plot_model

from transform import (
    CATEGORICAL_FEATURES,
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name,
)


def get_model(show_summary=True):
    """Define Keras model."""
    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )

    concat = tf.keras.layers.concatenate(input_features)
    deep = tf.keras.layers.Dense(256, activation="relu")(concat)
    deep = tf.keras.layers.Dense(64, activation="relu")(deep)
    deep = tf.keras.layers.Dense(16, activation="relu")(deep)

    num_classes = 4
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    if show_summary:
        model.summary()

    return model


def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        outputs = model(transformed_features)
        return {"outputs": outputs}

    return serve_tf_examples_fn


def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generates features and labels for tuning/training.
    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of
        returned dataset to combine in a single batch
    Returns:
        A dataset that contains (features, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset


# TFX Trainer will call this function.
def run_fn(fn_args):
    """Train the model based on given args.
    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 64)

    model = get_model()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback],
        epochs=10,
    )

    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

    plot_model(
        model, to_file="images/model_plot.png", show_shapes=True, show_layer_names=True
    )


Overwriting modules/trainer.py


Melakukan inisialisasi local pipeline

In [7]:
def init_local_pipeline(components, pipeline_root: Text) -> pipeline.Pipeline:
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args,
    )


Menjalankan pipeline menggunakan Apache Beam

In [8]:
from modules.components import init_components
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
logging.set_verbosity(logging.INFO)


components = init_components(
    DATA_ROOT,
    training_module=TRAINER_MODULE_FILE,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    training_steps=5000,
    eval_steps=1000,
    serving_model_dir=serving_model_dir,
)

# Jalankan komponen satu per satu
context = InteractiveContext(pipeline_root=pipeline_root)
for c in components:
        print(f"Running component: {c.id}")
        context.run(c)



INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'units_1', 'default': 128, 'conditions': [], 'values': [128, 256, 512], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'dropout_1', 'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'units_2', 'default': 64, 'conditions': [], 'values': [64, 128], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'dropout_2', 'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'units_3', 'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'dropout_3', 'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.0

Trial 10 Complete [00h 01m 20s]
val_sparse_categorical_accuracy: 1.0

Best val_sparse_categorical_accuracy So Far: 1.0
Total elapsed time: 00h 21m 47s
Results summary
Results in output\pilcotech-obesity-pipeline\.temp\6\tfx_tuner
Showing 10 best trials
Objective(name="val_sparse_categorical_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units_1: 512
dropout_1: 0.4
units_2: 128
dropout_2: 0.4
units_3: 64
dropout_3: 0.2
learning_rate: 0.001
Score: 1.0

Trial 01 summary
Hyperparameters:
units_1: 128
dropout_1: 0.3
units_2: 128
dropout_2: 0.3
units_3: 32
dropout_3: 0.2
learning_rate: 0.001
Score: 1.0

Trial 02 summary
Hyperparameters:
units_1: 128
dropout_1: 0.2
units_2: 64
dropout_2: 0.4
units_3: 32
dropout_3: 0.3
learning_rate: 0.001
Score: 1.0

Trial 04 summary
Hyperparameters:
units_1: 128
dropout_1: 0.2
units_2: 128
dropout_2: 0.4
units_3: 32
dropout_3: 0.3
learning_rate: 0.0001
Score: 1.0

Trial 07 summary
Hyperparameters:
units_1: 512
dropout_1: 0.3
units_2: 128
drop

INFO:absl:Successfully built user code wheel distribution at 'output\\pilcotech-obesity-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+c1f279af4dd3aece4ce778c5bdcf0aab5122d7518f65a6baf825d1e516f1ec85-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@output\\pilcotech-obesity-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+c1f279af4dd3aece4ce778c5bdcf0aab5122d7518f65a6baf825d1e516f1ec85-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 5000,\n  "splits": [\n    "train"\n  ]\n}', 'eval_args': '{\n  "num_steps": 1000,\n  "splits": [\n    "eval"\n  ]\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'trainer@output\\pilcotech-obesity-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+c1f279af4dd3aece4ce778c5bdcf0aab5122d7518f65a6baf825d1e516f1ec85

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Gender_xf (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 Height_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 Weight_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 BMI_xf (InputLayer)            [(None, 1)]          0           []                               
                                                                                            

INFO:tensorflow:struct2tensor is not available.


...

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Label_xf"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "Gender_xf"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairn

INFO:absl:Evaluation complete. Results written to output\pilcotech-obesity-pipeline\Evaluator\evaluation\9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output\pilcotech-obesity-pipeline\Evaluator\blessing\9.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1751855789
INFO:absl:Model written to serving path output\serving_model\1751855789.
INFO:absl:Model pushed to output\pilcotech-obesity-pipeline\Pusher\pushed_model\10.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


Running component: Pusher
